In [1]:
from sklearn.externals import joblib
import os
import random
import numpy
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.grid_search import GridSearchCV 

In [7]:
base_root='/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/targetfinder/GM12878/output-epw/'

random.seed(666)

fin1  = open(base_root+'training.csv','r')

pbin   = [ [],[],[],[],[] ]
nbin   = [ [],[],[],[],[] ]
bins   = []
neg    = []
pos    = []
for line in fin1:
	if line[0] == 'b':
		continue
	else:
		data  = line.split('"')
		if data[1] not in bins:
			bins.append(data[1])
		index = bins.index(data[1])
		data  = line.split(',')
		if data[7] == '1':
			pbin[index].append(line)
			pos.append(line)
		else:
			nbin[index].append(line)	


for i in range(len(bins)):
	nbinlen = len(nbin[i])
	temp    = random.sample(nbin[i],nbinlen/20)
	neg.extend(temp)

nnum   = len(neg)
pnum   = len(pos)
fnum   = len(neg[0].strip().split(',')) - 18

print nnum,pnum,fnum

fin2   = open(base_root+'enhancers.bed','r')
fin3   = open(base_root+'promoters.bed','r')
enhancers = []
promoters = []

arrays1 = numpy.zeros((nnum+pnum,100,2))
labels1 = numpy.zeros(nnum+pnum)

for line in fin2:
	data = line.strip().split()
	enhancers.append(data[3])

for line in fin3:
	data = line.strip().split()
	promoters.append(data[3])

enhancer_model = Doc2Vec.load(base_root+'enhancers_6_1_100.d2v')
promoter_model = Doc2Vec.load(base_root+'promoters_6_1_100.d2v')


for i in range(len(pos)):
        data = pos[i].strip().split(',')
        e_index = enhancers.index(data[5])
        p_index = promoters.index(data[10])
        prefix_enhancer = 'ENHANCERS_' + str(e_index)
        prefix_promoter = 'PROMOTERS_' + str(p_index)
        enhancer_vec = enhancer_model.docvecs[prefix_enhancer]
        promoter_vec = promoter_model.docvecs[prefix_promoter]
        enhancer_vec = enhancer_vec.reshape((100,1))
        promoter_vec = promoter_vec.reshape((100,1))
	arrays1[i] = numpy.append(enhancer_vec,promoter_vec,axis=1)
	labels1[i] = 1

for j in range(len(neg)):
        data = neg[j].strip().split(',')
        e_index = enhancers.index(data[5])
        p_index = promoters.index(data[10])
        prefix_enhancer = 'ENHANCERS_' + str(e_index)
        prefix_promoter = 'PROMOTERS_' + str(p_index)
        enhancer_vec = enhancer_model.docvecs[prefix_enhancer]
        promoter_vec = promoter_model.docvecs[prefix_promoter]
        enhancer_vec = enhancer_vec.reshape((100,1))
        promoter_vec = promoter_vec.reshape((100,1))  #.reshape((1,100))
	arrays1[i+j] = numpy.append(enhancer_vec,promoter_vec,axis=1)  #umn_stack
	labels1[i+j] = 0

print numpy.shape(arrays1)

arrays1 = numpy.array(arrays1)
labels1 = numpy.array(labels1)

In [11]:
from numpy import random
from keras.utils.np_utils import to_categorical
index = range(len(labels1))
random.shuffle(index)
SPLIT_RATIO = 0.85
split_point =int(SPLIT_RATIO*len(labels1))

print split_point

x_train , y_train=  arrays1[index[:split_point]] ,labels1[ index [:split_point]]
x_valid ,y_valid= arrays1[index[split_point:]] , labels1[index[split_point:]]

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

In [17]:
import pickle as pkl

In [21]:
savedata =open('./Gen_data/14_DocVec_all_data.pkl','w')
pkl.dump(x_train,savedata)
pkl.dump(y_train,savedata)
pkl.dump(x_valid,savedata)
pkl.dump(y_valid,savedata)


In [22]:
loaddata = open('./Gen_data/14_DocVec_all_data.pkl','r')
x_train=pkl.load(loaddata)
y_train=pkl.load(loaddata)
x_valid=pkl.load(loaddata)
y_valid=pkl.load(loaddata)

# Network

In [24]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [25]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten

# Net1: shallow Dense

kmer_input = Input(shape=(100,2), dtype='float32')
l_flatten= Flatten()(kmer_input)

l_dense_0 = Dense(64)(l_flatten)
l_drop_0 = Dropout(0.2)(l_dense_0)

l_dense_1 = Dense(32)(l_drop_0)
l_drop_1 = Dropout(0.3)(l_dense_1)

l_dense_2 = Dense(16)(l_drop_1)
l_drop_2 = Dropout(0.5)(l_dense_2)

preds = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_drop_2)
model  = Model (kmer_input,preds)

效果很差,acc只有0.58

# Net2: deep Dense

kmer_input = Input(shape=(100,2), dtype='float32')
l_flatten= Flatten()(kmer_input)

l_dense_0 = Dense(200)(l_flatten)
l_drop_0 = Dropout(0.2)(l_dense_0)

l_dense_1 = Dense(128)(l_drop_0)
l_drop_1 = Dropout(0.3)(l_dense_1)

l_dense_2 = Dense(64)(l_drop_1)
l_drop_2 = Dropout(0.5)(l_dense_2)

l_dense_3 = Dense(32)(l_drop_2)
l_drop_3 = Dropout(0.5)(l_dense_3)

l_dense_4 = Dense(16)(l_drop_3)
l_drop_4 = Dropout(0.5)(l_dense_4)

preds = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_drop_4)
model  = Model (kmer_input,preds)

效果也很查,acc 只有0.6

# Net3 : Att-BLSTM

In [47]:
kmer_input = Input(shape=(100,2), dtype='float32')

l_lstm =Bidirectional(LSTM(100,return_sequences=True))(kmer_input)

l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
#l_flatten = Flatten()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_att)
#l_dense = Dense(256)(l_att_drop)
l_dense_1 = Dense(16)(l_att_drop)


preds = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_dense_1)
model  = Model (kmer_input,preds)

效果很差 最好才0.57

# Net4: BLSTM+Dense

In [ ]:
kmer_input = Input(shape=(100,2), dtype='float32')
l_lstm =Bidirectional(LSTM(5,return_sequences=True))(kmer_input)

l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
l_flatten = Flatten()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_flatten)
l_dense = Dense(128)(l_att_drop)
l_dense_1 = Dense(32)(l_dense)
preds = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_dense_1)
model  = Model (kmer_input,preds)

# Compile&Train

In [48]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 100, 2)        0                                            
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 100, 200)      82400       input_9[0][0]                    
____________________________________________________________________________________________________
dropout_20 (Dropout)             (None, 100, 200)      0           bidirectional_2[0][0]            
____________________________________________________________________________________________________
attlayer_2 (AttLayer)            (None, 200)           200         dropout_20[0][0]                 
____________________________________________________

In [50]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), nb_epoch=100, batch_size=50)

Train on 3589 samples, validate on 634 samples
Epoch 1/100
3589/3589 [==============================] - 33s - loss: 0.3713 - acc: 0.5600 - val_loss: 0.3737 - val_acc: 0.5315
Epoch 2/100
3589/3589 [==============================] - 33s - loss: 0.3700 - acc: 0.5748 - val_loss: 0.3691 - val_acc: 0.5505
Epoch 3/100
3589/3589 [==============================] - 32s - loss: 0.3722 - acc: 0.5573 - val_loss: 0.3725 - val_acc: 0.5095
Epoch 4/100
3589/3589 [==============================] - 33s - loss: 0.3748 - acc: 0.5138 - val_loss: 0.3735 - val_acc: 0.5000
Epoch 5/100
3589/3589 [==============================] - 34s - loss: 0.3746 - acc: 0.5141 - val_loss: 0.3713 - val_acc: 0.5773
Epoch 6/100
3589/3589 [==============================] - 32s - loss: 0.3742 - acc: 0.5216 - val_loss: 0.3710 - val_acc: 0.5552
Epoch 7/100
3589/3589 [==============================] - 33s - loss: 0.3742 - acc: 0.5364 - val_loss: 0.3721 - val_acc: 0.5284
Epoch 8/100
3589/3589 [==============================] - 33s - l

In [39]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), nb_epoch=80, batch_size=50)

Train on 3589 samples, validate on 634 samples
Epoch 1/80
3589/3589 [==============================] - 0s - loss: 0.3569 - acc: 0.6456 - val_loss: 0.3716 - val_acc: 0.5883
Epoch 2/80
3589/3589 [==============================] - 0s - loss: 0.3551 - acc: 0.6481 - val_loss: 0.3676 - val_acc: 0.5883
Epoch 3/80
3589/3589 [==============================] - 0s - loss: 0.3545 - acc: 0.6498 - val_loss: 0.3671 - val_acc: 0.5694
Epoch 4/80
3589/3589 [==============================] - 0s - loss: 0.3560 - acc: 0.6570 - val_loss: 0.3680 - val_acc: 0.5694
Epoch 5/80
3589/3589 [==============================] - 0s - loss: 0.3554 - acc: 0.6431 - val_loss: 0.3760 - val_acc: 0.5757
Epoch 6/80
3589/3589 [==============================] - 0s - loss: 0.3568 - acc: 0.6514 - val_loss: 0.3724 - val_acc: 0.5647
Epoch 7/80
3589/3589 [==============================] - 0s - loss: 0.3568 - acc: 0.6537 - val_loss: 0.3722 - val_acc: 0.5852
Epoch 8/80
3589/3589 [==============================] - 0s - loss: 0.3555 - ac

# Result  
1) 缺少两层全连接, 只有输出全连接层,100次,精度0.62  
2) 输出全连接层,缺少Att,100次,精度0.62   
3) 正常组: 加Att ,精度0.53  